<a href="https://colab.research.google.com/github/DavoII/handwritten_letters_recognition/blob/master/CNN_FOR_KERAS_TUNNER.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [25]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [0]:
import numpy as np
from kerastuner import RandomSearch
from tensorflow.keras import optimizers
from tensorflow.keras.models import Sequential
from sklearn.model_selection import train_test_split
from keras.preprocessing.image import ImageDataGenerator
from kerastuner.engine.hyperparameters import HyperParameters
from tensorflow.keras.layers import Dense, Dropout, Activation, Flatten, Conv2D, MaxPooling2D

In [27]:
data = np.load('drive/My Drive/lowercase_agg_first_10/data.npy', encoding='bytes') 
labels = np.load('drive/My Drive/lowercase_agg_first_10/labels.npy', encoding='bytes') 
class_idx = np.load('drive/My Drive/lowercase_agg_first_10/label_classes.npy', encoding='bytes')

print(data.shape)
print(labels.shape)

(46016, 64, 64)
(46016, 10)


In [28]:
number_classes = len(class_idx)
print('number of classes', number_classes)

number of classes 10


In [0]:
x_train, x_test, y_train, y_test = train_test_split(
    data, labels,
    test_size=0.2,
    random_state=42,
    shuffle=True,
    stratify=labels
    )

x_train = x_train.astype('float32')
x_test = x_test.astype('float32')
y_train = y_train.astype('float32')
y_test = y_test.astype('float32')

x_train /= 255
x_test /= 255

In [0]:
(img_shape_w, img_shape_h) = x_train[0].shape
x_train = x_train.reshape(x_train.shape[0], img_shape_w, img_shape_h, 1)
x_test = x_test.reshape(x_test.shape[0], img_shape_w, img_shape_h, 1)

In [31]:
print('x_train', x_train.shape)
print('y_train', y_train.shape)
print('x_test', x_test.shape)
print('y_test', y_test.shape)

x_train (36812, 64, 64, 1)
y_train (36812, 10)
x_test (9204, 64, 64, 1)
y_test (9204, 10)


In [0]:
def build_model(hp):

  categories=number_classes

  ly1_neurons = hp.Int('conv_1_filter', min_value=128, max_value=512, step=16)
  ly2_neurons = hp.Int('conv_2_filter', min_value=128, max_value=512, step=16)
  ly3_neurons = hp.Int('conv_3_filter', min_value=128, max_value=512, step=16)

  kernel_size_1 = hp.Choice('conv_1_kernel', values=[3, 5, 7])
  kernel_size_2 = hp.Choice('conv_2_kernel', values=[3, 5, 7])
  kernel_size_3 = hp.Choice('conv_3_kernel', values=[3, 5, 7])

  ly1_dropout = hp.Float('dropout_1', min_value=0.0, max_value=0.5, default=0.25, step=0.05)
  ly2_dropout = hp.Float('dropout_2', min_value=0.0, max_value=0.5, default=0.25, step=0.05)
  ly3_dropout = hp.Float('dropout_3', min_value=0.0, max_value=0.5, default=0.25, step=0.05)

  pool_size_1 = (2, 2)
  pool_size_2 = (2, 2)
  pool_size_3 = (2, 2)

  model = Sequential()
  
  model.add(Conv2D(filters=ly1_neurons, kernel_size=kernel_size_1, input_shape=x_train.shape[1:]))
  model.add(Activation("relu"))
  model.add(MaxPooling2D(pool_size=pool_size_1))
  model.add(Dropout(ly1_dropout))

  model.add(Conv2D(filters=ly2_neurons, kernel_size=kernel_size_2))
  model.add(Activation("relu"))
  model.add(MaxPooling2D(pool_size=pool_size_2))
  model.add(Dropout(ly2_dropout))

  model.add(Conv2D(filters=ly3_neurons, kernel_size=kernel_size_3))
  model.add(Activation("relu"))
  model.add(MaxPooling2D(pool_size=pool_size_3))
  model.add(Dropout(ly3_dropout))

  model.add(Flatten())
  model.add(Dense( 512 ))
  model.add(Activation('relu'))
  model.add(Dense( categories ))
  model.add(Activation('softmax'))

  model.compile(
      optimizer=optimizers.Adam(hp.Choice('learning_rate', values=[1e-2, 1e-4])),
      loss='categorical_crossentropy',
      metrics=['accuracy']
      )

  return model


In [33]:
!pip install keras-tuner

In [38]:
tuner_search = RandomSearch(
    build_model,
    objective='val_accuracy',
    max_trials=5,
    )

INFO:tensorflow:Reloading Oracle from existing project ./untitled_project/oracle.json
INFO:tensorflow:Reloading Tuner from ./untitled_project/tuner0.json


In [39]:
tuner_search.search(x_train, y_train, epochs=5, validation_split=0.2)

INFO:tensorflow:Oracle triggered exit


In [36]:
model_from_tunner = tuner_search.get_best_models(num_models=1)[0]


Two checkpoint references resolved to different objects (<tensorflow.python.keras.layers.core.Dense object at 0x7fc9a68bd7b8> and <tensorflow.python.keras.layers.core.Flatten object at 0x7fc9a68bd710>).


ValueError: ignored

In [0]:
model_from_tunner.summary()

In [0]:
model_from_tunner.fit(
    x_train,
    y_train,
    batch_size=120,
    epochs=10,
    validation_data=(x_test, y_test),
    verbose=1
    )